In [12]:
import numpy as np
from tkinter import Tk, Canvas
import time

# Simulation parameters
N_particles = 100  # Number of particles
L = 140  # Box size
x_min, x_max, y_min, y_max = -L / 2, L / 2, -L / 2, L / 2
dt = 0.01  # Time step
door_width = 5 # Width of the evacuation doors
exit_threshold = 5  # Distance threshold to consider a particle evacuated
speed = 5.0  # Speed of particles
door_capacity = 1  # Maximum number of particles that can pass through a door at once

# Door positions
door_1_x = x_min  # Left door x-position
door_1_y_min, door_1_y_max = -door_width / 2, door_width / 2  # Left door y-range
door_2_x = x_max  # Right door x-position
door_2_y_min, door_2_y_max = -door_width / 2, door_width / 2  # Right door y-range

# Initial positions and target doors
x = np.random.uniform(x_min, x_max, N_particles)
y = np.random.uniform(y_min, y_max, N_particles)
target_door = np.random.choice([1, 2], size=N_particles)

# Evacuation tracking
evacuated_particles = np.zeros(N_particles, dtype=bool)
door_usage = {"door_1": 0, "door_2": 0}
door_usage_T = {"door_1": 0, "door_2": 0}
# Set up visualization
window_size = 600
tk = Tk()
tk.geometry(f"{window_size + 20}x{window_size + 20}")
canvas = Canvas(tk, background="#ECECEC")
canvas.place(x=10, y=10, height=window_size, width=window_size)

# Draw walls with door gaps
wall_thickness = 5
canvas.create_rectangle(0, 0, window_size, wall_thickness, fill="black")  # Top wall
canvas.create_rectangle(
    0, window_size - wall_thickness, window_size, window_size, fill="black"
)  # Bottom wall
canvas.create_rectangle(0, 0, wall_thickness, (door_1_y_min + L / 2) / L * window_size, fill="black")  # Left wall
canvas.create_rectangle(0, (door_1_y_max + L / 2) / L * window_size, wall_thickness, window_size, fill="black")  # Left door gap
canvas.create_rectangle(
    window_size - wall_thickness, 0, window_size, (door_2_y_min + L / 2) / L * window_size, fill="black"
)  # Right wall
canvas.create_rectangle(
    window_size - wall_thickness,
    (door_2_y_max + L / 2) / L * window_size,
    window_size,
    window_size,
    fill="black",
)  # Right door gap

# Draw particles
particles = []
for j in range(N_particles):
    particles.append(
        canvas.create_oval(
            (x[j] - 1) / L * window_size + window_size / 2,
            (y[j] - 1) / L * window_size + window_size / 2,
            (x[j] + 1) / L * window_size + window_size / 2,
            (y[j] + 1) / L * window_size + window_size / 2,
            fill="#00C0C0",
        )
    )

# Helper functions
def move_toward_target(px, py, door):
    if door == 1:
        dx = door_1_x - px
        dy = (door_1_y_min + door_1_y_max) / 2 - py
    else:
        dx = door_2_x - px
        dy = (door_2_y_min + door_2_y_max) / 2 - py
    dist = np.sqrt(dx**2 + dy**2)
    return (speed * dx / dist, speed * dy / dist) if dist > 0 else (0, 0)

def check_overlap(idx):
    for j in range(N_particles):
        if j != idx and not evacuated_particles[j]:
            dist = np.sqrt((x[idx] - x[j])**2 + (y[idx] - y[j])**2)
            if dist < 2:
                return True
    return False

def update_particle_positions():
    global x, y, evacuated_particles, door_usage
    door_usage["door_1"] = 0
    door_usage["door_2"] = 0
    for i in range(N_particles):
        if not evacuated_particles[i]:
            vx, vy = move_toward_target(x[i], y[i], target_door[i])
            x[i] += vx * dt
            y[i] += vy * dt

            # Check for door usage
            if target_door[i] == 1 and door_usage["door_1"] < door_capacity and x[i] < x_min + exit_threshold and door_1_y_min <= y[i] <= door_1_y_max:
                evacuated_particles[i] = True
                door_usage["door_1"] += 1
                door_usage_T["door_1"] += 1
            elif target_door[i] == 2 and door_usage["door_2"] < door_capacity and x[i] > x_max - exit_threshold and door_2_y_min <= y[i] <= door_2_y_max:
                evacuated_particles[i] = True
                door_usage["door_2"] += 1
                door_usage_T["door_2"] += 1

# Simulation loop
step = 0
while not np.all(evacuated_particles):
    update_particle_positions()

    for j, particle in enumerate(particles):
        if not evacuated_particles[j]:
            canvas.coords(
                particle,
                (x[j] - 1) / L * window_size + window_size / 2,
                (y[j] - 1) / L * window_size + window_size / 2,
                (x[j] + 1) / L * window_size + window_size / 2,
                (y[j] + 1) / L * window_size + window_size / 2,
            )
        else:
            canvas.coords(particle, 0, 0, 0, 0)  # Hide evacuated particles

    tk.update()
    time.sleep(0.00001)
    step += 1

# Output total evacuation time
print(f"Total evacuation time: {step * dt:.2f} seconds")
print(f"door_1: {door_usage_T["door_1"]} ")
print(f"door_2: {door_usage_T["door_2"]} ")
tk.mainloop()


Total evacuation time: 29.68 seconds
door_1: 54 
door_2: 46 
